In [ ]:
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import auc, f1_score, accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
with open('train.json') as data_file:    
    data = json.load(data_file)

In [ ]:
products_num = np.array([len(i["ingredients"]) for i in data])

In [ ]:
cuisine = []
ingredients = []
ingredients_num = []
for i in data:
    if len(i["ingredients"]) < 50:
        ingredients.append(" ".join(i["ingredients"]))
        cuisine.append(i["cuisine"])
        ingredients_num.append(len(i["ingredients"]))

In [ ]:
df = pd.DataFrame()
df["cuisine"] = cuisine
df["ingridients"] = ingredients
df["ingredients_num"] = ingredients_num

In [ ]:
(df.groupby("cuisine")["ingredients_num"].mean().reset_index()
                                         .sort_values("ingredients_num", ascending=False))

In [ ]:
plt.figure(figsize=(20, 10))
chart = sns.countplot(cuisine)
ch = chart.set_xticklabels(chart.get_xticklabels(), rotation=65, horizontalalignment='right')

In [ ]:
vectorizer = TfidfVectorizer(min_df=10, max_df=0.5)

In [ ]:
X = vectorizer.fit_transform(ingredients)

In [ ]:
print(X.shape)

In [ ]:
LE = LabelEncoder()
labels = LE.fit_transform(cuisine)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.33, random_state=42)

In [ ]:
def print_metrics(y_test, ans):
    print "Accuracy ~", np.round(accuracy_score(y_test, ans), 4)
    print "F1_score ~", np.round(f1_score(y_test, ans, average="macro"), 4)

# Stacking

# LR

In [ ]:
LR = LogisticRegression(solver='liblinear')
LR.fit(X_train, y_train)

In [ ]:
lr_ans = LR.predict(X_test)

In [ ]:
print_metrics(y_test, lr_ans)

# RF

In [ ]:
RF = RandomForestClassifier(n_estimators=100)
RF.fit(X_train, y_train.ravel())

In [ ]:
RF_ans = RF.predict(X_test)

In [ ]:
print_metrics(y_test, RF_ans)

In [ ]:
X_train.shape

In [ ]:
LR.predict_proba(X_train).shape

In [ ]:
RF_stack = RandomForestClassifier(n_estimators=100)
RF_stack.fit(LR.predict_proba(X_train), y_train.ravel())

In [ ]:
RF_predict_proba = RF_stack.predict(LR.predict_proba(X_test))

In [ ]:
RF_ans_stack = RF_stack.predict(np.hstack((X_test.toarray(), LR.predict_proba(X_test))))

In [ ]:
print_metrics(y_test, RF_predict_proba)

In [ ]:
print_metrics(y_test, RF_ans_stack)

# Bootstrap

In [ ]:
df["cuisine"].value_counts()

In [ ]:
cuisine1 = "russian"
cuisine2 = "brazilian"

In [ ]:
cuisine1_df = df[df["cuisine"] == cuisine1]
cuisine2_df = df[df["cuisine"] == cuisine2]

In [ ]:
plt.figure(figsize=(16, 8))
plt.title("Ingridients number in {} cuisine".format(cuisine1))
sns.countplot(cuisine1_df["ingredients_num"])

plt.figure(figsize=(16, 8))
plt.title("Ingridients number in {} cuisine".format(cuisine2))
sns.countplot(cuisine2_df["ingredients_num"])

In [ ]:
plt.figure(figsize=(16, 8))
fig = sns.kdeplot(cuisine1_df["ingredients_num"], label = cuisine1)
fig = sns.kdeplot(cuisine2_df["ingredients_num"], label = cuisine2)        
fig.set(xlabel=u'Количество ингридиентов', ylabel=u'Плотность')    
plt.grid()
plt.show()

Теперь было бы хорошо оценить, из скольки ингридентов в среднем состоят блюда каждой из кухонь. Так как данных в нашем датасете мало, то искать среднее не совсем правильно, лучше применить наши новые знания бутстрэпа.

In [ ]:
def get_bootstrap_samples(data, n_samples):
    """Функция для генерации n_samples подвыборок с помощью бутстрэпа"""
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [ ]:
def stat_intervals(stat, alpha):
    """Функция для интервальной оценки"""
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return np.round(boundaries, 4)

In [ ]:
cuisine1_ingridients = cuisine1_df["ingredients_num"].values
cuisine2_ingridients = cuisine2_df["ingredients_num"].values

In [ ]:
np.random.seed(42)

In [ ]:
cuisine1_ingridients_scores = [np.mean(sample) 
                       for sample in get_bootstrap_samples(cuisine1_ingridients, 5000)]
cuisine2_ingridients_scores = [np.mean(sample) 
                       for sample in get_bootstrap_samples(cuisine2_ingridients, 5000)]

In [ ]:
print "{} ingridients number:  mean interval".format(cuisine1),  stat_intervals(cuisine1_ingridients_scores, 0.05)
print "{} ingridients number:  mean interval".format(cuisine2),  stat_intervals(cuisine2_ingridients_scores, 0.05)

In [ ]:
print("""В итоге мы получили, что с 95% вероятностью среднее число ингридиентов в {} кухне будет лежать в промежутке между {r[0]} и {r[1]},  в то время как в {} в среднем от {b[0]} до {b[1]}"""
.format(cuisine1, cuisine2, r=stat_intervals(cuisine1_ingridients_scores, 0.05), b=stat_intervals(cuisine2_ingridients_scores, 0.05)))

# Бэггинг

In [ ]:
DT = DecisionTreeClassifier()
DT.fit(X_train, y_train)

In [ ]:
DT_ans = DT.predict(X_test)

In [ ]:
print_metrics(y_test, DT_ans)

# Bagging

In [ ]:
bagging_dt_answer = [np.bincount(answer[:, i]).argmax() for i in range(answer.shape[1])]

In [ ]:
print_metrics(y_test, bagging_dt_answer)

# RSM 

In [ ]:
rsm_dt_answer = [np.bincount(answer[:, i]).argmax() for i in range(answer.shape[1])]

In [ ]:
print_metrics(y_test, rsm_dt_answer)

# Random Forest

In [ ]:
class RandomForest():
    
    def __init__(self, n_estimators=20, max_depth=None, random_state=42):
            
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.trees_ = [] # 
        self.features_idx = []
        self.random_state = random_state

    
    def fit(self, X, y):
        self.X = X
        self.y = y
        for t in tqdm_notebook(range(self.n_estimators)):               
            # выбираем базовый алгоритм - дерево
            
            # сэмплируем объекты
            
            # сэмплируем признаки

            # обучаем

            # добавляем алгоритм к ансамблю

            # добавляем признаки

                   
        return self
    
    def predict(self, X):
        answer = np.zeros((len(self.trees_), X.shape[0]), dtype=int)
        # добавляем прогнозы деревьев
        for t in range(len(self.trees_)):
            answer[t] = 
            
        return np.array([np.bincount(answer[:, i]).argmax() for i in range(answer.shape[1])])

In [ ]:
RF = RandomForest()
RF.fit(X_train, y_train)

In [ ]:
RF_ans = RF.predict(X_test)

In [ ]:
print_metrics(y_test, RF_ans)

In [ ]:
RF_stack.feature_importances_

In [ ]:
RF_stack.feature_importances_[-20:]

In [ ]:
np.argsort(RF_stack.feature_importances_)[-20:]

In [ ]:
np.argsort(RF_stack.feature_importances_)[-20:] >= 1489

# Ограничим датасет для быстроты подбора параметров

british          804
filipino         755
irish            667
jamaican         526
russian          489
brazilian        466

In [ ]:
cutted_labels = LE.transform(["british", "filipino", "irish", "jamaican", "russian", "brazilian", "spanish"])

In [ ]:
idx = []
for i in range(len(labels)):
    if labels[i] in cutted_labels:
        idx.append(i)
idx = np.array(idx)

In [ ]:
X, y = X[idx], labels[idx]

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
RFC = RandomForestClassifier(n_estimators=100, random_state=42, oob_score=True)
temp_train_acc = []
temp_test_acc = []
temp_train_f1 = []
temp_test_f1 = []
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    RFC.fit(X_train, y_train)
    temp_train_acc.append(accuracy_score(RFC.predict(X_train), y_train))
    temp_train_f1.append(f1_score(RFC.predict(X_train), y_train, average="macro"))
    temp_test_acc.append(accuracy_score(RFC.predict(X_test), y_test))
    temp_test_f1.append(f1_score(RFC.predict(X_test), y_test, average="macro"))

In [ ]:
print "f1_score is", np.round(np.mean(temp_test_f1), 5)
print "accuracy is", np.round(np.mean(temp_test_acc), 5)

In [ ]:
def visualize(test_metric_acc, train_metric_acc, test_metric_f1, train_metric_f1, grid, xlabel='X'):

    train_acc, test_acc = np.asarray(train_metric_acc), np.asarray(test_metric_acc)
    train_f1, test_f1 = np.asarray(train_metric_f1), np.asarray(test_metric_f1)
    print "Best accuracy_score on CV is {:.4f} with {}".format(max(test_acc.mean(axis=1)), 
                                                    grid[np.argmax(test_acc.mean(axis=1))]), xlabel
    plt.style.use('ggplot')
    plt.figure(figsize=(16, 8))
    plt.title("Accuracy")
    plt.plot(grid, test_acc.mean(axis=1), label="test")
    plt.plot(grid, train_acc.mean(axis=1), label="train")
    plt.xlabel(xlabel)
    plt.legend()
    plt.show()
    
    print "Best f1_score on CV is {:.4f} with {}".format(max(test_f1.mean(axis=1)), 
                                                    grid[np.argmax(test_f1.mean(axis=1))]), xlabel
    
    plt.style.use('ggplot')
    plt.figure(figsize=(16, 8))
    plt.title("F1_score")
    plt.plot(grid, test_f1.mean(axis=1), label="test")
    plt.plot(grid, train_f1.mean(axis=1), label="train")
    plt.xlabel(xlabel)
    plt.legend()
    plt.show()

In [ ]:
def CV(RFC, X, y):

    temp_train_acc = []
    temp_test_acc = []
    temp_train_f1 = []
    temp_test_f1 = []
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        RFC.fit(X_train, y_train)
        train_ans = RFC.predict(X_train)
        test_ans = RFC.predict(X_test)
        temp_train_acc.append(accuracy_score(y_train, train_ans))
        temp_train_f1.append(f1_score(y_train, train_ans, average="macro"))
        temp_test_acc.append(accuracy_score(y_test, test_ans))
        temp_test_f1.append(f1_score(y_test, test_ans, average="macro"))
    
    return temp_train_acc, temp_test_acc, temp_train_f1, temp_test_f1

# Количество деревьев в ансамбле

In [ ]:
train_acc = []
test_acc = []
train_f1 = []
test_f1 = []
trees_grid = [5, 10, 15, 20, 30, 50, 75, 100]

for ntrees in tqdm_notebook(trees_grid):
    RFC = RandomForestClassifier(n_estimators=ntrees, random_state=42)
    temp_train_acc, temp_test_acc, temp_train_f1, temp_test_f1 = CV(RFC, X, y)
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    train_f1.append(temp_train_f1)
    test_f1.append(temp_test_f1)

In [ ]:
visualize(test_acc, train_acc, test_f1, train_f1, trees_grid, "trees number")

# Глубина леса

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

train_acc = []
test_acc = []
train_f1 = []
test_f1 = []
max_depth_grid = [5, 10, 15, 20, 30, 50, 100, 150]

for max_depth in tqdm_notebook(max_depth_grid):
    RFC = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=max_depth)
    temp_train_acc, temp_test_acc, temp_train_f1, temp_test_f1 = CV(RFC, X, y)
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    train_f1.append(temp_train_f1)
    test_f1.append(temp_test_f1)

In [ ]:
visualize(test_acc, train_acc, test_f1, train_f1, max_depth_grid, "max_depth")

# MAX_FEATURES

##### По умолчанию он равен sqrt(n) в задачах классификации и n/3 в задачах регрессии

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

train_acc = []
test_acc = []
train_f1 = []
test_f1 = []
max_features_grid = [5, 10, 15, 20, 38, 50, 100, 500, 1000]

for max_features in tqdm_notebook(max_features_grid):
    RFC = RandomForestClassifier(n_estimators=100, random_state=42, max_features=max_features)
    temp_train_acc, temp_test_acc, temp_train_f1, temp_test_f1 = CV(RFC, X, y)
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    train_f1.append(temp_train_f1)
    test_f1.append(temp_test_f1)

In [ ]:
visualize(test_acc, train_acc, test_f1, train_f1, max_features_grid, "max_features")

# MIN SAMPLES LEAF

По классике, в задачах регрессии рекомендуется использовать значение 5

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

train_acc = []
test_acc = []
train_f1 = []
test_f1 = []
min_samples_leaf_grid = [1, 3, 5, 7, 10, 15, 20, 25]

for min_samples_leaf in tqdm_notebook(min_samples_leaf_grid):
    RFC = RandomForestClassifier(n_estimators=100, random_state=42, min_samples_leaf=min_samples_leaf)
    temp_train_acc, temp_test_acc, temp_train_f1, temp_test_f1 = CV(RFC, X, y)
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    train_f1.append(temp_train_f1)
    test_f1.append(temp_test_f1)

In [ ]:
visualize(test_acc, train_acc, test_f1, train_f1, min_samples_leaf_grid, "min_samples_leaf_grid")

In [ ]:
np.sqrt(X.shape[1])

In [ ]:
parameters = {'max_features': [1, 5, 10, 15, 20, 38, 50], 
              'min_samples_leaf': [1, 3, 5, 7],
              'max_depth': [5, 10, 15, 20, 30, 50]}
rfc = RandomForestClassifier(n_estimators=10, random_state=42, 
                             n_jobs=18, oob_score=True)
gcv = GridSearchCV(rfc, parameters, n_jobs=18, cv=skf, verbose=1)
gcv.fit(X, y)

In [ ]:
gcv.best_estimator_

In [ ]:
gcv.best_score_

# Градиентный Бустинг

# Для простоты рассмотрим регрессию

In [ ]:
from sklearn.metrics import mean_squared_error as mse

In [ ]:
wine = pd.read_csv("winequality-red.csv")

In [ ]:
X = wine.values[:, :-1]
y = wine.values[:, -1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(wine.values[:, :-1], wine.values[:, -1], test_size=0.33, random_state=42)

$MSE(y, p) = \frac{1}{n} (y-p)^T(y-p) = \frac{1}{n} \sum\limits_{i=1}^n(y_i - p_i)^2$

$\nabla MSE - ?$

Каждый следующий алгоритм тоже будем настраивать на остатки предыдущих

Заметим, что остатки могут быть найдены как антиградиент функции потерь по ответу модели, посчитанный в точке ответа уже построенной композиции

In [ ]:
class GradientBoosting():
    
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=-1, random_state=42):
            
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.learning_rate = learning_rate
        self.initialization = lambda y: np.mean(y) * np.ones(len(y))
        self.loss_by_iter = [] # функция потерь на каждой итерации
        self.trees_ = []
        self.random_state = random_state
        
    def mse_grad(self, y, p):
        # написать градиент
        return 

    
    def fit(self, X, y):
        self.X = X
        self.y = y
        b = self.initialization(y)
        prediction = b.copy()
        for t in tqdm_notebook(range(self.n_estimators)):               
            # считаем - антиградиент
            resid = 
            # выбираем базовый алгоритм
            tree = 
            # обучаемся на векторе антиградиента
            tree.fit
            # делаем предикт и добавляем алгоритм к ансамблю
            b = tree
            #добавляем дерево в ансамбль
            self.trees_
            # обновляем текущее приближение (lr * b)
            prediction +=
            
            # обновляем лосс на обучении (опционально)
            self.loss_by_iter
            
        return self
    
    def predict(self, X):
        # сначала инициализируем прогноз на тестовой выборке – 
        # это просто вектор из средних значений ответов на обучении
        pred = 
        # добавляем прогнозы деревьев * lr
        for t in range(self.n_estimators):
            pred +=
            
        return pred

In [ ]:
tree = GradientBoosting(n_estimators=500, learning_rate=0.01, max_depth=None)
tree.fit(X_train, y_train)
mse(y_test, tree.predict(X_test))

In [ ]:
tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
mse(y_test, tree.predict(X_test))

In [ ]:
tree = GradientBoosting(n_estimators=500, learning_rate=1., max_depth=None)
tree.fit(X_train, y_train)
mse(y_test, tree.predict(X_test))

In [ ]:
class StohasticGradientBoosting():
    


In [ ]:
tree = GradientBoosting(n_estimators=500, learning_rate=0.9, max_depth=None)
tree.fit(X_train, y_train)
mse(y_test, tree.predict(X_test))

In [ ]:
from lightgbm import LGBMRegressor, LGBMClassifier

LGBM = LGBMRegressor(n_estimators=500)
LGBM.fit(X_train, y_train.ravel())
LGBM_ans = LGBM.predict(X_test)
mse(y_test, LGBM_ans)

In [ ]:
def visualize(test_metric_mse, train_metric_mse, grid, xlabel='X'):

    train_mse, test_mse = np.asarray(train_metric_mse), np.asarray(test_metric_mse)
    print "Best MSE on CV is {:.4f} with {}".format(min(test_mse.mean(axis=1)), 
                                                    grid[np.argmin(test_mse.mean(axis=1))]), xlabel
    plt.style.use('ggplot')
    plt.figure(figsize=(16, 8))
    plt.title("MSE")
    plt.plot(grid, test_mse.mean(axis=1), label="test")
    plt.plot(grid, train_mse.mean(axis=1), label="train")
    plt.xlabel(xlabel)
    plt.legend()
    plt.show()

# Параметры Градиентного бустинга

# Learning Rate

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

train_mse = []
test_mse = []
learning_rates = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 1.]

for lr in tqdm_notebook(learning_rates):
    LGBM = LGBMRegressor(learning_rate=lr)
    temp_train_mse = []
    temp_test_mse = []
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        LGBM.fit(X_train, y_train)
        temp_train_mse.append(mse(LGBM.predict(X_train), y_train))
        temp_test_mse.append(mse(LGBM.predict(X_test), y_test))
    train_mse.append(temp_train_mse)
    test_mse.append(temp_test_mse)

In [ ]:
visualize(test_mse, train_mse, learning_rates, "learning rate")

# n_estimators

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

train_mse = []
test_mse = []
n_estimators = [1, 2, 4, 8, 16, 32, 64, 100, 200]

for n in tqdm_notebook(n_estimators):
    LGBM = LGBMRegressor(n_estimators=n)
    temp_train_mse = []
    temp_test_mse = []
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        LGBM.fit(X_train, y_train)
        temp_train_mse.append(mse(LGBM.predict(X_train), y_train))
        temp_test_mse.append(mse(LGBM.predict(X_test), y_test))
    train_mse.append(temp_train_mse)
    test_mse.append(temp_test_mse)

In [ ]:
visualize(test_mse, train_mse, n_estimators, "n_estimators")